# Environment Prep

This notebook sits between:
- tf20_vision_samples/tf2x Obect Detection tutorial
- tensorflow/models/research/object detection  -- which you cannot update

## NOTES - troubleshooting
- the tutorial is written for Windows (ewww) - and, that means '\' in the paths - you need to change them

In [1]:
import os
import sys

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [3]:
print (tf.__version__)

2.3.0


### Globals

In [4]:
# TODO - put logic in here to computer
HOME = os.path.expanduser("~")
PROJECTS = os.path.join(HOME, "projects")
MODELS = os.path.join(HOME, "projects/tensorflow/models")
MODEL_RESEARCH = os.path.join(MODELS, "research")
OBJ_DETECT = os.path.join(MODELS, "research/object_detection")


TRAINING_DEMO = os.path.join(PROJECTS, "tensorflow/workspace/training_demo")
PIPELINE_CONFIG = os.path.join(TRAINING_DEMO, "models/my_ssd_resnet50_v1_fpn","laptop_pipeline.config")
TRAINED_EXPORT = os.path.join(TRAINING_DEMO, "exported-models/")
INFERENCE_GRAPH = os.path.join(TRAINING_DEMO, "inference_graph")

In [5]:
!ls {TRAINING_DEMO}/inference_graph

## DATA

you only need to move the data once - it will be there between restarts

### tfrecords
the tutorial says you need 2 new files under the training_demo/annotations folder, named 
- test.record
- train.record

In [6]:
import os

In [ ]:
!pwd

In [ ]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/annotations

## Model
download the ResNet50 640x640 model  

copy the pipeline.config file to my* folder.   Then edit locally with gedit.

In [ ]:
# you should see checkpoint, pipeline.config, saved_model
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/

In [ ]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn -l

## TF 2.x GPU
Common Problems
- if list_physical_devices shows GPU and gpu_device_name is none, then you probably have a CUDA version issue - check CUDA version, PATH & LD_LIBRARY path in ./bashrc.     e.g.   path needs to include CUDA 10.1 for TF2.2
- check your version of TensorFlow, one of the steps automatically upgrades from TF 2.x to 2.4.   Thus, you need CUDA 11.0 installed.   However, I didn't notice any path requirements...

In [ ]:
import tensorflow as tf

In [ ]:
print (tf.__version__)

In [ ]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
physical_devices= tf.config.experimental.list_physical_devices('GPU')
print(len(physical_devices))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

## Tutorial - Object Detection API Setup

In [ ]:
!pip install pycocotools

In [ ]:
import pycocotools

### API

commom problems:

#### ASUS


#### Dell Inspiron

```ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paramiko 2.4.2 requires cryptography>=1.5, which is not installed.
fabric 2.4.0 requires cryptography>=1.1, which is not installed.
aws 0.2.5 requires boto, which is not installed.
tensorflow-gpu 2.2.0 requires tensorboard<2.3.0,>=2.2.0, but you have tensorboard 2.4.0 which is incompatible.
tensorflow-gpu 2.2.0 requires tensorflow-estimator<2.3.0,>=2.2.0, but you have tensorflow-estimator 2.4.0 which is incompatible.

```
- pip install crytography  # 3.3.1
- pip install boto # 2.49.0
- DON'T mess with tensorflow* dependencies - because this script is in some kind of loop!   I had 2.2 then it uninstalled and upgraded to 2.4 so the fact that it is complaining about 2.4 is kind of bogus.   I suppose you could back it down and it would probably work.  BUT - I did nothing and then reran the script and it worked fine

Successfully installed object-detection-0.1

In [ ]:
original_cwd = cwd = os.getcwd()
print ("Original working directory:", original_cwd)

In [ ]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/models/research/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
# make sure your kernel is correct:  conda_tensorflow2_p36
!python -m pip install .

### Test the API
you should be in models/research

Possible Problems:
- GPU error - memory;   you may need to close this notebook and others to free up GPU memory

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
os.chdir(original_cwd)
cwd = os.getcwd() 
print("back to original working directory:", cwd)

## train

speed benchmarks
- SageMaker - 27.7 minutes!! per step - that was CPU only becuase CUDA 11.0 is not installed
- ASUS 1080 TI - 0.7 sec (8 images/batch)
- Dell Inspiron 1060 - 1.0  (4 images/batch, memory warnings)

In [ ]:
!cp /home/ec2-user/SageMaker/tensorflow/models/research/object_detection/model_main_tf2.py .

In [ ]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/workspace/training_demo/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

## export

https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb

In [ ]:
!ls {HOME}/projects/tensorflow/models/research/object_detection/exp*.py

In [ ]:
os.chdir(TRAINING_DEMO)
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
# my_ssd_resnet50_v1_fpn
# LAPTOP config
!ls
!python ./exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/laptop_pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/resnet50_640

In [ ]:
!ls {TRAINING_DEMO}/exported-models/resnet50_640
!ls {TRAINING_DEMO}/exported-models/resnet50_640/checkpoint -l
!ls {TRAINING_DEMO}/exported-models/resnet50_640/saved_model -l

In [ ]:
# no $ in the command line
# add $ in the passed variables?

!python {MODEL_RESEARCH}/object_detection/export_inference_graph.py \
  --input_type image_tensor \
  --pipeline_config_path="{PIPELINE_CONFIG}" \ \
  --trained_checkpoint_prefix="{TRAINED_EXPORT}/ckpt-0" \
  --output_directory="{INFERENCE_GRAPH}"

In [12]:
#path of the directory where you want to save your model
frozen_out_path = INFERENCE_GRAPH
# name of the .pb file
frozen_graph_filename = "frozen_graph"
model_path = '/home/jay/projects/tensorflow/workspace/training_demo/exported-models/resnet50_640/saved_model'

loaded_model = tf.keras.models.load_model('/home/jay/projects/tensorflow/workspace/training_demo/exported-models/resnet50_640/saved_model/')

# tf.keras.models.load_model(model_path, custom_objects=None, compile=True, options=None)

IndexError: list index out of range

In [8]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

NameError: name 'model' is not defined

In [9]:
!ls {model_path} -l
model = tf.compat.v2.saved_model.load(export_dir=model_path)
print (type(model))

total 13232
drwxr-xr-x 2 jay jay     4096 Dec 21 10:41 assets
-rw-r--r-- 1 jay jay 13538598 Dec 21 10:41 saved_model.pb
drwxr-xr-x 2 jay jay     4096 Dec 21 10:41 variables
<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>


In [10]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

AttributeError: '_UserObject' object has no attribute 'inputs'

In [11]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_tensor'] tensor_info:
        dtype: DT_UINT8
        shape: (1, -1, -1, 3)
        name: serving_default_input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['detection_anchor_indices'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 100)
        name: StatefulPartitionedCall:0
    outputs['detection_boxes'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 100, 4)
        name: StatefulPartitionedCall:1
    outputs['de